In [1]:
# Initial imports
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
from libs.drew_lib import *
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
asset_dataframe_list = []
for asset in keywords:

    asset_dataframe = pd.read_csv(f'./Data/Cleaned_Data/test/{asset}.csv', parse_dates = True, infer_datetime_format = True)
    asset_dataframe = asset_dataframe.set_index('Unnamed: 0')
    asset_dataframe.index.name = None
    cols = ['stockmarket_positive_sentiment',
        'stockmarket_negative_sentiment',
        'stockmarket_neutral_sentiment',	
        'stockmarket_compound_sentiment',
        'volume',
        'close'
    ]
    asset_dataframe = asset_dataframe[cols]
    asset_dataframe.rename(columns={
        'stockmarket_compound_sentiment':'r/stockmarket_compound',
        'stockmarket_positive_sentiment':'r/stockmarket_pos',
        'stockmarket_neutral_sentiment':'r/stockmarket_neu',
        'stockmarket_negative_sentiment':'r/stockmarket_neg'
    },inplace=True)

    asset_dataframe_list.append(asset_dataframe)

In [12]:
asset_sec_list = []
for asset in keywords:

    asset_dataframe = pd.read_csv(f'./Data/Cleaned_Data/SEC_sentiment_and_stocks/{asset}.csv', parse_dates = True, infer_datetime_format = True)
    asset_dataframe = asset_dataframe.set_index('date')
    asset_dataframe.index.name = None
    asset_dataframe.drop(asset_dataframe.columns[-3:],axis=1,inplace=True)
    asset_dataframe.rename(columns={
        'pos':'sec_pos',
        'neg':'sec_neg',
        'neu':'sec_neu',
        'compound':'sec_compund'
    },inplace=True)

    asset_sec_list.append(asset_dataframe)

In [16]:
NFLX = pd.concat([asset_sec_list[0],asset_dataframe_list[0]],axis=1,join='outer')
FB = pd.concat([asset_sec_list[1],asset_dataframe_list[1]],axis=1,join='outer')
UBER = pd.concat([asset_sec_list[2],asset_dataframe_list[2]],axis=1,join='outer')
MCHP = pd.concat([asset_sec_list[3],asset_dataframe_list[3]],axis=1,join='outer')
ABNB = pd.concat([asset_sec_list[4],asset_dataframe_list[4]],axis=1,join='outer')
FANG = pd.concat([asset_sec_list[5],asset_dataframe_list[5]],axis=1,join='outer')
MRO = pd.concat([asset_sec_list[6],asset_dataframe_list[6]],axis=1,join='outer')
DVN = pd.concat([asset_sec_list[7],asset_dataframe_list[7]],axis=1,join='outer')
SPWR = pd.concat([asset_sec_list[8],asset_dataframe_list[8]],axis=1,join='outer')
REGI = pd.concat([asset_sec_list[9],asset_dataframe_list[9]],axis=1,join='outer')
MTRX = pd.concat([asset_sec_list[10],asset_dataframe_list[10]],axis=1,join='outer')
BLK = pd.concat([asset_sec_list[11],asset_dataframe_list[11]],axis=1,join='outer')
PYPL = pd.concat([asset_sec_list[12],asset_dataframe_list[12]],axis=1,join='outer')
MELI = pd.concat([asset_sec_list[13],asset_dataframe_list[13]],axis=1,join='outer')
SOFI = pd.concat([asset_sec_list[14],asset_dataframe_list[14]],axis=1,join='outer')

In [24]:
def fill_and_bin(df):
    df = return_bin(df)
    df['return_bin'] = df['return_bin'].fillna('no gain/loss')
    df['returns'] = df['returns'].fillna(0)
    df[['r/stockmarket_compound','r/stockmarket_pos','r/stockmarket_neu','r/stockmarket_neg','close']] = df[['r/stockmarket_compound','r/stockmarket_pos','r/stockmarket_neu','r/stockmarket_neg','close']].fillna(method='ffill')
    df['volume'] = df['volume'].fillna(df['volume'].mean())
    return df

In [ ]:
ticker_df_list = [NFLX,FB,UBER,MCHP,ABNB,FANG,MRO, DVN, SPWR,REGI,MTRX,BLK, PYPL,MELI,SOFI]
[fill_and_bin(df) for df in ticker_df_list]